# Implementing a Multi-Tier Architecture with FastAPI

In this tutorial, we'll explore how to design and implement a **multi-tier architecture** using **FastAPI**. We'll cover the fundamental concepts of tiered architecture, explain the roles of different layers, and demonstrate how to structure your FastAPI application accordingly. By the end of this tutorial, you'll understand how to build scalable, maintainable, and modular applications using FastAPI.

## Table of Contents

1. [Introduction](#1-introduction)
2. [Prerequisites](#2-prerequisites)
3. [Understanding Multi-Tier Architecture](#3-understanding-multi-tier-architecture)
   - [3.1. Single-Tier Architecture](#31-single-tier-architecture)
   - [3.2. Two-Tier Architecture](#32-two-tier-architecture)
   - [3.3. Three-Tier Architecture](#33-three-tier-architecture)
   - [3.4. N-Tier Architecture](#34-n-tier-architecture)
4. [Benefits of Tiered Architecture](#4-benefits-of-tiered-architecture)
5. [Setting Up the Environment](#5-setting-up-the-environment)
6. [Designing the Application Layers](#6-designing-the-application-layers)
   - [6.1. Presentation Layer](#61-presentation-layer)
   - [6.2. Business Logic Layer](#62-business-logic-layer)
   - [6.3. Data Access Layer](#63-data-access-layer)
   - [6.4. Database Layer](#64-database-layer)
7. [Implementing the Application](#7-implementing-the-application)
   - [7.1. Project Structure](#71-project-structure)
   - [7.2. Defining the Models](#72-defining-the-models)
   - [7.3. Creating the Data Access Layer](#73-creating-the-data-access-layer)
   - [7.4. Implementing the Business Logic Layer](#74-implementing-the-business-logic-layer)
   - [7.5. Building the Presentation Layer](#75-building-the-presentation-layer)
8. [Connecting the Layers](#8-connecting-the-layers)
9. [Implementing Dependency Injection](#9-implementing-dependency-injection)
10. [Handling Errors and Exceptions](#10-handling-errors-and-exceptions)
11. [Testing the Application](#11-testing-the-application)
12. [Conclusion](#12-conclusion)
13. [References](#13-references)

## 1. Introduction

Tiered architecture, also known as **multi-tier architecture**, is a client-server architecture in which presentation, application processing, and data management functions are physically separated. This separation allows developers to manage complexity, improve scalability, and facilitate maintenance.

In this tutorial, we'll:

- Understand the different tiers in software architecture.
- Design a FastAPI application following a multi-tier architecture.
- Implement each layer with clear separation of concerns.
- Connect the layers using dependency injection.
- Handle errors and test the application effectively.

## 2. Prerequisites

Before we begin, ensure you have the following:

- **Python 3.7+** installed.
- Basic knowledge of **Python**, **FastAPI**, and **SQLAlchemy**.
- Familiarity with concepts like **asynchronous programming** and **dependency injection**.

## 3. Understanding Multi-Tier Architecture

### 3.1. Single-Tier Architecture

- **Monolithic Application**: All components are combined into a single layer.
- **Pros**: Simple to develop and deploy.
- **Cons**: Difficult to scale and maintain as the application grows.

### 3.2. Two-Tier Architecture

- **Client-Server Model**: Presentation layer (client) communicates directly with the data layer (server/database).
- **Pros**: Improved scalability over single-tier.
- **Cons**: Business logic is often mixed with presentation or data layers.

### 3.3. Three-Tier Architecture

- **Layers**:
  - **Presentation Layer**: User interface.
  - **Business Logic Layer**: Application logic.
  - **Data Access Layer**: Interacts with the database.
- **Pros**: Separation of concerns, easier maintenance.
- **Cons**: Slightly increased complexity.

### 3.4. N-Tier Architecture

- **Additional Layers**: Security, caching, logging, etc.
- **Pros**: Highly scalable and maintainable.
- **Cons**: Increased complexity and potential performance overhead.

## 4. Benefits of Tiered Architecture

- **Separation of Concerns**: Each layer handles specific responsibilities.
- **Maintainability**: Easier to update or replace individual layers.
- **Scalability**: Layers can be scaled independently.
- **Reusability**: Components can be reused across different parts of the application.
- **Testability**: Layers can be tested in isolation.

## 5. Setting Up the Environment

Create a new project directory and set up a virtual environment.

```bash
# Create project directory
mkdir fastapi-tiered-architecture
cd fastapi-tiered-architecture

# Set up virtual environment
python -m venv venv
source venv/bin/activate  # On Windows: venv\Scripts\activate
```

Install the necessary packages.

```bash
pip install fastapi uvicorn sqlalchemy asyncpg pydantic[dotenv] alembic
```

## 6. Designing the Application Layers

We'll design an application that manages a simple product catalog. The application will be structured into the following layers:

### 6.1. Presentation Layer

- **Role**: Handles HTTP requests and responses.
- **Components**: FastAPI endpoints, request parsing, response formatting.

### 6.2. Business Logic Layer

- **Role**: Contains the core application logic.
- **Components**: Services, validation, business rules.

### 6.3. Data Access Layer

- **Role**: Interacts with the database.
- **Components**: Repositories, data models, CRUD operations.

### 6.4. Database Layer

- **Role**: Stores the data.
- **Components**: Database server (e.g., PostgreSQL).

## 7. Implementing the Application

### 7.1. Project Structure

```
fastapi-tiered-architecture/
├── app/
│   ├── __init__.py
│   ├── main.py
│   ├── core/
│   │   ├── __init__.py
│   │   ├── config.py
│   │   └── database.py
│   ├── models/
│   │   ├── __init__.py
│   │   └── product.py
│   ├── repositories/
│   │   ├── __init__.py
│   │   └── product_repository.py
│   ├── services/
│   │   ├── __init__.py
│   │   └── product_service.py
│   ├── schemas/
│   │   ├── __init__.py
│   │   └── product.py
│   └── api/
│       ├── __init__.py
│       └── endpoints/
│           ├── __init__.py
│           └── product.py
├── alembic/
│   └── ... (Alembic configuration files)
├── requirements.txt
```

### 7.2. Defining the Models

**app/models/product.py**

```python
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import declarative_base

Base = declarative_base()

class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String(255), index=True)
    description = Column(String(255))
    price = Column(Float)
```

### 7.3. Creating the Data Access Layer

**app/repositories/product_repository.py**

```python
from sqlalchemy.ext.asyncio import AsyncSession
from sqlalchemy.future import select
from typing import List, Optional

from app.models.product import Product

class ProductRepository:
    def __init__(self, session: AsyncSession):
        self.session = session

    async def get(self, product_id: int) -> Optional[Product]:
        result = await self.session.execute(select(Product).where(Product.id == product_id))
        return result.scalars().first()

    async def get_all(self) -> List[Product]:
        result = await self.session.execute(select(Product))
        return result.scalars().all()

    async def create(self, product: Product) -> Product:
        self.session.add(product)
        await self.session.commit()
        await self.session.refresh(product)
        return product

    async def update(self, product: Product) -> Product:
        self.session.add(product)
        await self.session.commit()
        await self.session.refresh(product)
        return product

    async def delete(self, product: Product):
        await self.session.delete(product)
        await self.session.commit()
```

### 7.4. Implementing the Business Logic Layer

**app/services/product_service.py**

```python
from typing import List

from app.repositories.product_repository import ProductRepository
from app.models.product import Product
from app.schemas.product import ProductCreate, ProductUpdate

class ProductService:
    def __init__(self, product_repository: ProductRepository):
        self.product_repository = product_repository

    async def get_product(self, product_id: int) -> Product:
        product = await self.product_repository.get(product_id)
        if not product:
            raise ValueError("Product not found")
        return product

    async def get_products(self) -> List[Product]:
        return await self.product_repository.get_all()

    async def create_product(self, product_in: ProductCreate) -> Product:
        product = Product(
            name=product_in.name,
            description=product_in.description,
            price=product_in.price
        )
        return await self.product_repository.create(product)

    async def update_product(self, product_id: int, product_in: ProductUpdate) -> Product:
        product = await self.get_product(product_id)
        product.name = product_in.name or product.name
        product.description = product_in.description or product.description
        product.price = product_in.price or product.price
        return await self.product_repository.update(product)

    async def delete_product(self, product_id: int):
        product = await self.get_product(product_id)
        await self.product_repository.delete(product)
```

### 7.5. Building the Presentation Layer

**app/api/endpoints/product.py**

```python
from fastapi import APIRouter, Depends, HTTPException, status
from typing import List

from app.schemas.product import ProductCreate, ProductRead, ProductUpdate
from app.services.product_service import ProductService
from app.dependencies import get_product_service

router = APIRouter()

@router.get("/", response_model=List[ProductRead])
async def read_products(product_service: ProductService = Depends(get_product_service)):
    return await product_service.get_products()

@router.get("/{product_id}", response_model=ProductRead)
async def read_product(product_id: int, product_service: ProductService = Depends(get_product_service)):
    try:
        return await product_service.get_product(product_id)
    except ValueError as e:
        raise HTTPException(status_code=404, detail=str(e))

@router.post("/", response_model=ProductRead, status_code=status.HTTP_201_CREATED)
async def create_product(product_in: ProductCreate, product_service: ProductService = Depends(get_product_service)):
    return await product_service.create_product(product_in)

@router.put("/{product_id}", response_model=ProductRead)
async def update_product(product_id: int, product_in: ProductUpdate, product_service: ProductService = Depends(get_product_service)):
    try:
        return await product_service.update_product(product_id, product_in)
    except ValueError as e:
        raise HTTPException(status_code=404, detail=str(e))

@router.delete("/{product_id}", status_code=status.HTTP_204_NO_CONTENT)
async def delete_product(product_id: int, product_service: ProductService = Depends(get_product_service)):
    try:
        await product_service.delete_product(product_id)
    except ValueError as e:
        raise HTTPException(status_code=404, detail=str(e))
    return None
```

**app/main.py**

```python
from fastapi import FastAPI
from app.api.endpoints import product

app = FastAPI()

app.include_router(product.router, prefix="/products", tags=["products"])
```


## 8. Connecting the Layers

To connect the layers, we'll use dependency injection to provide instances of repositories and services to the components that need them.

**app/dependencies.py**

```python
from sqlalchemy.ext.asyncio import AsyncSession, create_async_engine
from sqlalchemy.orm import sessionmaker
from typing import AsyncGenerator

from app.core.config import settings
from app.services.product_service import ProductService
from app.repositories.product_repository import ProductRepository

DATABASE_URL = settings.DATABASE_URL

engine = create_async_engine(DATABASE_URL, echo=True)
AsyncSessionLocal = sessionmaker(bind=engine, class_=AsyncSession, expire_on_commit=False)

async def get_db_session() -> AsyncGenerator[AsyncSession, None]:
    async with AsyncSessionLocal() as session:
        yield session

async def get_product_repository(session: AsyncSession = Depends(get_db_session)) -> ProductRepository:
    return ProductRepository(session)

async def get_product_service(product_repository: ProductRepository = Depends(get_product_repository)) -> ProductService:
    return ProductService(product_repository)
```

**Explanation**:

- **get_db_session**: Provides an asynchronous database session.
- **get_product_repository**: Provides an instance of `ProductRepository`.
- **get_product_service**: Provides an instance of `ProductService`.

## 9. Implementing Dependency Injection

Dependency injection allows us to manage dependencies between components. FastAPI's `Depends` function facilitates this.

**Usage in Presentation Layer**:

```python
from fastapi import Depends

# In your endpoint functions
async def read_products(product_service: ProductService = Depends(get_product_service)):
    # Use product_service
```

This way, FastAPI automatically resolves the dependencies and provides the required instances.

## 10. Handling Errors and Exceptions

We can create custom exceptions and error handlers to manage errors gracefully.

**app/core/exceptions.py**

```python
class NotFoundException(Exception):
    def __init__(self, name: str):
        self.name = name
```

**app/main.py**

```python
from fastapi import Request
from fastapi.responses import JSONResponse
from app.core.exceptions import NotFoundException

@app.exception_handler(NotFoundException)
async def not_found_exception_handler(request: Request, exc: NotFoundException):
    return JSONResponse(
        status_code=404,
        content={"detail": f"{exc.name} not found"}
    )
```

**Usage in Services**:

```python
from app.core.exceptions import NotFoundException

async def get_product(self, product_id: int) -> Product:
    product = await self.product_repository.get(product_id)
    if not product:
        raise NotFoundException("Product")
    return product
```

## 11. Testing the Application

We can test the application using **pytest** and **httpx**.

**test_main.py**

```python
import pytest
from httpx import AsyncClient
from app.main import app

@pytest.mark.asyncio
async def test_read_products():
    async with AsyncClient(app=app, base_url="http://test") as ac:
        response = await ac.get("/products/")
    assert response.status_code == 200
    assert isinstance(response.json(), list)
```

**Running the Tests**:

```bash
pytest test_main.py
```

## 12. Conclusion

In this tutorial, we've:

- Understood the principles of multi-tier architecture.
- Designed a FastAPI application with clear separation of concerns.
- Implemented each layer: Presentation, Business Logic, Data Access, and Database.
- Connected the layers using dependency injection.
- Handled errors and exceptions gracefully.
- Tested the application to ensure it works as expected.

By structuring your application in tiers, you enhance its scalability, maintainability, and testability.

## 13. References

- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [SQLAlchemy Documentation](https://docs.sqlalchemy.org/en/14/)
- [Pydantic Documentation](https://pydantic-docs.helpmanual.io/)
- [Dependency Injection in FastAPI](https://fastapi.tiangolo.com/tutorial/dependencies/)
- [Asynchronous SQLAlchemy](https://docs.sqlalchemy.org/en/14/orm/extensions/asyncio.html)
- [Testing FastAPI Applications](https://fastapi.tiangolo.com/tutorial/testing/)